In [1]:
%cd ..

C:\Users\wuyua\PycharmProjects\RxExperiments


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from matplotlib import pyplot as plt
from rx import operators

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration()

In [3]:
import logging
from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)
logging_output

Output()

In [4]:
from controls.modbus_pump import ModbusPumpControl, ModbusPumpControlCommand

pump_control = ModbusPumpControl()

In [5]:
import ipywidgets as widgets
import rx
from utils.observable_factory import from_callback

slurry_input = widgets.FloatText(description="Slurry: ")
clear_input = widgets.FloatText(description="Clear: ")
        
slurry_obs = from_callback(slurry_input.observe)().pipe(
    operators.filter(lambda x: x["type"] == "change"),
    operators.map(lambda x: x["new"]),
    operators.filter(lambda x: isinstance(x, float)),
)
clear_obs = from_callback(clear_input.observe)().pipe(
    operators.filter(lambda x: x["type"] == "change"),
    operators.map(lambda x: x["new"]),
    operators.filter(lambda x: isinstance(x, float)),
)

rx.combine_latest(slurry_obs, clear_obs).pipe(
    operators.map(lambda x: ModbusPumpControlCommand().set_clear_pump(x[1]).set_slurry_pump(x[0]))
).subscribe(pump_control)
widgets.HBox([slurry_input, clear_input])